In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from collections import Counter
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from dotenv import dotenv_values
config = dotenv_values('.env')

In [2]:
insta_ID = config['insta_ID']
insta_PD = config['insta_PD']

In [10]:
## 검색, 인기게시물
main_hashtag = "dog"
max_hashtag = 20

post_count_dic = {}

In [25]:
def wait_for(locator):
    return WebDriverWait(browser, 4).until(
    EC.presence_of_element_located(locator)
    )

In [26]:
def login():
    
    browser.get(f'https://www.instagram.com/accounts/login/')
    
    wait_for((By.CLASS_NAME, "_2hvTZ"))

    insta_id = browser.find_element_by_name('username')
    insta_pw = browser.find_element_by_name('password')

    insta_id.send_keys(insta_ID)
    insta_pw.send_keys(insta_PD)
    insta_pw.send_keys(Keys.ENTER)

    time.sleep(3)

In [27]:
def extract_data():
    for i in browser.window_handles:
        next_tab = i
        browser.switch_to.window(next_tab)
        hashtag = wait_for((By.CLASS_NAME, '_7UhW9')).text[1:]
        posts_count = wait_for((By.CLASS_NAME, 'g47SY')).text.replace(',','')
        post_count_dic[hashtag] = posts_count

In [28]:
def get_related(search_url):
    login()
    browser.get(search_url)
    time.sleep(1)
    pop_tag_li = poppost2tag(browser, frequency=10)
    
    ## 많이 나온 태그들 다시 검색해서 새탭에서 열기
    for tag in pop_tag_li:
        search_url = f'https://www.instagram.com/explore/tags/{tag}'
        browser.execute_script("window.open('" +search_url +"');")
        
    extract_data()
    
    if len(post_count_dic) < max_hashtag:
        for window in browser.window_handles[0:-1]:
            browser.switch_to.window(window)
            browser.close()
        browser.switch_to.window(browser.window_handles[0])
        get_related(browser.current_url)
    

def poppost2tag(browser, frequency):
    pop_posts = browser.find_elements_by_class_name('_9AhH0')[:9]
    pop_posts_tags = []

    for post in pop_posts:
        post.click()
        try:
            wait_for((By.CLASS_NAME, "xil3i"))
            tag_list = browser.find_elements_by_class_name('xil3i')
            for tag in tag_list:
                pop_posts_tags.append(tag.text[1:])
        except:
            pass
        ActionChains(browser).send_keys(Keys.ESCAPE).perform()
    ## 인기게시물들의 태그들 중 빈도수 순 10개 가져오기
    pop_tags = Counter(pop_posts_tags).most_common(frequency)
    pop_tag_li = [i for i,c in pop_tags]
    pop_tag_li.remove(main_hashtag)
    return pop_tag_li

In [29]:
browser = webdriver.Chrome(ChromeDriverManager().install())
get_related(f'https://www.instagram.com/explore/tags/{main_hashtag}')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\guswn\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


TimeoutException: Message: 


In [4]:
## 로그인
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(f'https://www.instagram.com/accounts/login/')

wait_for((By.CLASS_NAME, "_2hvTZ"))

insta_id = browser.find_element_by_name('username')
insta_pw = browser.find_element_by_name('password')

insta_id.send_keys(insta_ID)
insta_pw.send_keys(insta_PD)
insta_pw.send_keys(Keys.ENTER)

time.sleep(3)




search_url = f'https://www.instagram.com/explore/tags/{main_hashtag}'

browser.get(search_url)
time.sleep(1)

pop_posts = browser.find_elements_by_class_name('_9AhH0')[:9]

# print(len(pop_posts))


## 인기게시물들에 있는 태그 다 가져오기
pop_posts_tags = []

for post in pop_posts:
    post.click()
    try:
        wait_for((By.CLASS_NAME, "xil3i"))
        tag_list = browser.find_elements_by_class_name('xil3i')
        for tag in tag_list:
            pop_posts_tags.append(tag.text[1:])
    except:
        pass
    ActionChains(browser).send_keys(Keys.ESCAPE).perform()
    

# print(len(pop_posts_tags))


## 인기게시물들의 태그들 중 빈도수 순 10개 가져오기
pop_tags = Counter(pop_posts_tags).most_common(10)
pop_tag_li = [i for i,c in pop_tags]
pop_tag_li.remove(main_hashtag)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\guswn\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


9
277


In [5]:
## 많이 나온 태그들 다시 검색해서 새탭에서 열기
for tag in pop_tag_li:
    main_hashtag = tag
    search_url = f'https://www.instagram.com/explore/tags/{main_hashtag}'
    browser.execute_script("window.open('" +search_url +"');")

len(browser.window_handles)

10

In [6]:
## 모든 탭의 게시물 수 가져오기
post_count_dic = {}

for i in browser.window_handles:
    next_tab = i
    browser.switch_to.window(next_tab)
    hashtag = wait_for((By.CLASS_NAME, '_7UhW9')).text[1:]
    posts_count = wait_for((By.CLASS_NAME, 'g47SY')).text.replace(',','')
    post_count_dic[hashtag] = posts_count
    
    

post_count_dic

{'dog': '309032289',
 'dogstagram': '108272135',
 'dogoftheday': '61422827',
 'dogsofinstagram': '242499541',
 'doglover': '66599306',
 'puppy': '161840725',
 'puppiesofinstagram': '52034277',
 'puppies': '42376667',
 'puppylove': '66876849',
 'dogs': '133010188'}

In [7]:
for window in browser.window_handles[0:-1]:
    browser.switch_to.window(window)
    browser.close()

In [ ]:
#         ActionChains(browser).key_down(Keys.CONTROL).click(tag).perform()

In [ ]:
class instamining():
    
    def __init__(self, main_hashtag):
        self.self.browser = webdriver.Chrome(ChromeDriverManager().install())
        self.main_hashtag = main_hashtag
        
    def login(self):
        self.browser.get(f'https://www.instagram.com/accounts/login/')

        WebDriverWait(self.browser, 3).until(
        EC.presence_of_element_located((By.CLASS_NAME, "_2hvTZ"))
        )

        insta_id = self.browser.find_element_by_name('username')
        insta_pw = self.browser.find_element_by_name('password')

        insta_id.send_keys(insta_ID)
        insta_pw.send_keys(insta_PD)
        insta_pw.send_keys(Keys.ENTER)

        time.sleep(3)

        
    def search(self, main_hashtag):
        self.browser.get(f'https://www.instagram.com/explore/tags/{main_hashtag}')
        time.sleep(1)
        header = WebDriverWait(browser,3).until(
            EC.presence_of_element_located((By.TAG_NAME, "header"))
        )

        self.get_pop_tags()
        

    def get_pop_tags(self):
        pop_posts = self.browser.find_elements_by_class_name('_9AhH0')[:9]
        pop_posts_tags = []
        for post in pop_posts:
            post.click()
            try:
                WebDriverWait(self.browser, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "xil3i"))
                )
                tag_list = self.browser.find_elements_by_class_name('xil3i')
                for tag in tag_list:
                    pop_posts_tags.append(tag.text[1:])
            except:
                pass
            ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
            
    def 
        self.pop_tags = Counter(pop_posts_tags).most_common(10)
        for tag in pop_tags:
            self.search(tag)

    

In [56]:
# pop_tags = ko.vocab().most_common(10)

# ko = nltk.Text(pop_posts_tags, name='태그')

# plt.figure(figsize=(12,6))
# ko.plot(50)

# wordcloud = WordCloud(relative_scaling=0.2, background_color='white',
#                      ).generate_from_frequencies(dict(data))

# plt.figure(figsize=(12,8))
# plt.imshow(wordcloud)

In [ ]:
# from wordcloud import WordCloud, STOPWORDS
# import matplotlib.pyplot as plt
# import nltk